# 04 - Chatbots Básicos

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 1.5-2 horas

---

## Índice

1. [Introducción a los Chatbots](#intro)
2. [Chatbot con reglas simples](#reglas)
3. [Chatbot con LLM (API gratuita)](#llm)
4. [Añadiendo memoria a la conversación](#memoria)
5. [Interfaces con Streamlit](#streamlit)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender los diferentes tipos de chatbots
- Construir un chatbot básico con reglas
- Integrar un LLM para respuestas inteligentes
- Implementar memoria conversacional
- Crear interfaces de usuario con Streamlit

<a name="intro"></a>
## 1. Introducción a los Chatbots

### Tipos de chatbots

| Tipo | Descripción | Ejemplo |
|------|-------------|--------|
| **Basado en reglas** | Respuestas predefinidas, keywords | FAQ bots simples |
| **Retrieval-based** | Busca respuestas en base de conocimiento | Customer support |
| **Generativo (LLM)** | Genera respuestas dinámicamente | ChatGPT, Claude |
| **Híbrido** | Combina reglas + LLM | Asistentes empresariales |

### Arquitectura básica de un chatbot

```
Usuario → [Input] → [Procesamiento] → [Generación] → [Output] → Usuario
                          ↑
                     [Memoria/Contexto]
```

In [1]:
# Install required libraries
!pip install -q langchain langchain-groq langchain-community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.4 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.33.5 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.20.0 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
from getpass import getpass

# Setup Groq API (FREE tier)
if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("Introduce tu GROQ API Key: ")

print("API Key configurada ✓")

Introduce tu GROQ API Key:  ········


API Key configurada ✓


<a name="reglas"></a>
## 2. Chatbot con reglas simples

El chatbot más simple utiliza coincidencia de palabras clave para seleccionar respuestas predefinidas.

In [3]:
class RuleBasedChatbot:
    """Simple rule-based chatbot using keyword matching."""
    
    def __init__(self):
        # Define rules as keyword -> response mappings
        self.rules = {
            'hola': '¡Hola! ¿En qué puedo ayudarte?',
            'buenos días': '¡Buenos días! ¿Cómo estás?',
            'adiós': '¡Hasta luego! Que tengas un buen día.',
            'gracias': '¡De nada! ¿Necesitas algo más?',
            'precio': 'Nuestros precios varían según el producto. ¿Cuál te interesa?',
            'horario': 'Nuestro horario es de 9:00 a 18:00, de lunes a viernes.',
            'contacto': 'Puedes contactarnos en info@ejemplo.com o al 900 123 456.',
            'ayuda': 'Puedo ayudarte con: precios, horarios, contacto. ¿Qué necesitas?'
        }
        self.default_response = "Lo siento, no entendí tu pregunta. ¿Puedes reformularla?"
    
    def respond(self, user_input: str) -> str:
        """Generate response based on keyword matching."""
        user_input_lower = user_input.lower()
        
        # Check each rule
        for keyword, response in self.rules.items():
            if keyword in user_input_lower:
                return response
        
        return self.default_response

# Test the rule-based chatbot
bot = RuleBasedChatbot()

test_messages = [
    "Hola, ¿qué tal?",
    "¿Cuál es el precio del producto?",
    "¿Cuál es su horario?",
    "Quiero comprar un coche",  # No matching rule
    "Gracias por la ayuda"
]

print("Chatbot basado en reglas:")
print("=" * 50)
for msg in test_messages:
    response = bot.respond(msg)
    print(f"👤 Usuario: {msg}")
    print(f"🤖 Bot: {response}\n")

Chatbot basado en reglas:
👤 Usuario: Hola, ¿qué tal?
🤖 Bot: ¡Hola! ¿En qué puedo ayudarte?

👤 Usuario: ¿Cuál es el precio del producto?
🤖 Bot: Nuestros precios varían según el producto. ¿Cuál te interesa?

👤 Usuario: ¿Cuál es su horario?
🤖 Bot: Nuestro horario es de 9:00 a 18:00, de lunes a viernes.

👤 Usuario: Quiero comprar un coche
🤖 Bot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?

👤 Usuario: Gracias por la ayuda
🤖 Bot: ¡De nada! ¿Necesitas algo más?



### Limitaciones del chatbot basado en reglas

| ✅ Ventajas | ❌ Desventajas |
|------------|---------------|
| Rápido y predecible | No entiende contexto |
| Sin costos de API | Respuestas limitadas |
| Fácil de mantener | No maneja variaciones |
| Control total | No aprende |

<a name="llm"></a>
## 3. Chatbot con LLM (API gratuita)

Ahora crearemos un chatbot más inteligente usando un LLM a través de Groq (tier gratuito).

In [4]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

class LLMChatbot:
    """Chatbot powered by LLM (using free Groq API)."""
    
    def __init__(self, system_prompt: str = None):
        self.llm = ChatGroq(
            model_name="llama-3.3-70b-versatile",
            temperature=0.7
        )
        
        # Default system prompt
        self.system_prompt = system_prompt or """Eres un asistente amable y servicial.
        Responde de forma concisa y clara.
        Si no sabes algo, dilo honestamente."""
    
    def respond(self, user_input: str) -> str:
        """Generate response using LLM."""
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=user_input)
        ]
        
        response = self.llm.invoke(messages)
        return response.content

# Test the LLM chatbot
llm_bot = LLMChatbot()

test_messages = [
    "Hola, ¿qué tal?",
    "¿Puedes explicarme qué es machine learning en términos simples?",
    "Quiero comprar un coche, ¿qué consejos me das?"
]

print("Chatbot con LLM:")
print("=" * 50)
for msg in test_messages:
    print(f"👤 Usuario: {msg}")
    response = llm_bot.respond(msg)
    print(f"🤖 Bot: {response}\n")

Chatbot con LLM:
👤 Usuario: Hola, ¿qué tal?
🤖 Bot: ¡Hola! Estoy bien, gracias. ¿En qué puedo ayudarte hoy?

👤 Usuario: ¿Puedes explicarme qué es machine learning en términos simples?
🤖 Bot: ¡Claro! El **machine learning** (aprendizaje automático) es una técnica que permite a las computadoras "aprender" de los datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente.

**Funciona de la siguiente manera:**

1. Se proporciona a la computadora un conjunto de datos (como imágenes, textos o números).
2. La computadora busca patrones y relaciones en los datos.
3. A partir de estos patrones, la computadora crea un modelo que puede predecir o clasificar nuevos datos.

**Ejemplos:**

* Un sistema de recomendación de películas que sugiere películas basadas en tus gustos previos.
* Un software de reconocimiento de voz que aprende a entender tu acento y vocabulario.

En resumen, el machine learning es como enseñar a una computadora a aprender de los datos y tomar decis

### Chatbot especializado con rol

In [5]:
# Create a specialized chatbot for tech support
tech_support_prompt = """Eres un técnico de soporte experto en productos tecnológicos.
Tu empresa vende:
- Laptops (garantía 2 años)
- Smartphones (garantía 1 año)  
- Tablets (garantía 1 año)

Horario de soporte: Lunes a Viernes, 9:00-18:00
Email: soporte@techstore.com
Teléfono: 900 111 222

Responde de forma profesional pero amigable.
Si el problema requiere atención presencial, recomienda visitar una tienda."""

tech_bot = LLMChatbot(system_prompt=tech_support_prompt)

tech_questions = [
    "Mi laptop no enciende, ¿qué puedo hacer?",
    "¿Cuánto dura la garantía de los smartphones?",
    "¿Tienen servicio de reparación los fines de semana?"
]

print("Chatbot de Soporte Técnico:")
print("=" * 50)
for q in tech_questions:
    print(f"👤 Usuario: {q}")
    response = tech_bot.respond(q)
    print(f"🤖 Bot: {response}\n")

Chatbot de Soporte Técnico:
👤 Usuario: Mi laptop no enciende, ¿qué puedo hacer?
🤖 Bot: Lo siento mucho que estés experimentando problemas con tu laptop. Antes de nada, te recomiendo verificar algunos puntos básicos para asegurarnos de que no se trata de un problema simple de resolución inmediata.

1. **Carga la batería**: Asegúrate de que la batería de tu laptop esté completamente cargada o, si es posible, intenta conectarla directamente a la corriente eléctrica.
2. **Verifica el cable de alimentación**: Revisa el cable de alimentación para asegurarte de que no esté dañado y que esté conectado correctamente tanto a la laptop como a la toma de corriente.
3. **Botón de encendido**: Asegúrate de que estás presionando el botón de encendido correcto y de la manera adecuada.

Si después de verificar estos puntos, tu laptop sigue sin encenderse, es posible que se trate de un problema más serio que requiera atención técnica especializada.

**Opciones para seguir:**

- **Garantía**: Si tu lapto

<a name="memoria"></a>
## 4. Añadiendo memoria a la conversación

Un chatbot sin memoria no puede mantener una conversación coherente. Vamos a implementar memoria conversacional.

In [6]:
class ChatbotWithMemory:
    """Chatbot with conversation memory."""
    
    def __init__(self, system_prompt: str = None):
        self.llm = ChatGroq(
            model_name="llama-3.3-70b-versatile",
            temperature=0.7
        )
        
        self.system_prompt = system_prompt or "Eres un asistente amable y servicial."
        
        # Initialize conversation history
        self.history = []
    
    def respond(self, user_input: str) -> str:
        """Generate response while maintaining conversation history."""
        # Build messages with history
        messages = [SystemMessage(content=self.system_prompt)]
        
        # Add conversation history
        for human_msg, ai_msg in self.history:
            messages.append(HumanMessage(content=human_msg))
            messages.append(AIMessage(content=ai_msg))
        
        # Add current message
        messages.append(HumanMessage(content=user_input))
        
        # Get response
        response = self.llm.invoke(messages)
        ai_response = response.content
        
        # Save to history
        self.history.append((user_input, ai_response))
        
        return ai_response
    
    def clear_history(self):
        """Clear conversation history."""
        self.history = []
        print("Historial borrado.")

# Test chatbot with memory
memory_bot = ChatbotWithMemory()

print("Chatbot con memoria:")
print("=" * 50)

# Conversation that requires memory
conversation = [
    "Hola, me llamo Carlos.",
    "¿Cuál es la capital de Francia?",
    "¿Y cuántos habitantes tiene?",
    "¿Cómo me llamo?"  # Test if bot remembers the name
]

for msg in conversation:
    print(f"👤 Usuario: {msg}")
    response = memory_bot.respond(msg)
    print(f"🤖 Bot: {response}\n")

Chatbot con memoria:
👤 Usuario: Hola, me llamo Carlos.
🤖 Bot: **Hola Carlos**

Me alegra conocerte. Soy un asistente amable y servicial, aquí para ayudarte con cualquier pregunta o tema que desees discutir. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para escucharte y proporcionarte la mejor ayuda posible.

👤 Usuario: ¿Cuál es la capital de Francia?
🤖 Bot: **La capital de Francia es París**

París es una de las ciudades más emblemáticas y famosas del mundo, conocida por sus monumentos históricos como la Torre Eiffel, el Louvre y Notre Dame. Es un destino turístico muy popular y un centro cultural y económico importante en Europa.

¿Quieres saber más sobre París o Francia? Estoy aquí para ayudarte con cualquier pregunta que tengas.

👤 Usuario: ¿Y cuántos habitantes tiene?
🤖 Bot: **Población de París**

Según los datos más recientes, la ciudad de París tiene una población de aproximadamente **2,1 millones de habitantes**. Sin e

In [7]:
# Show conversation history
print("Historial de la conversación:")
print("=" * 50)
for i, (human, ai) in enumerate(memory_bot.history, 1):
    print(f"Turno {i}:")
    print(f"  👤 Human: {human[:50]}..." if len(human) > 50 else f"  👤 Human: {human}")
    print(f"  🤖 AI: {ai[:50]}..." if len(ai) > 50 else f"  🤖 AI: {ai}")
    print()

Historial de la conversación:
Turno 1:
  👤 Human: Hola, me llamo Carlos.
  🤖 AI: **Hola Carlos**

Me alegra conocerte. Soy un asist...

Turno 2:
  👤 Human: ¿Cuál es la capital de Francia?
  🤖 AI: **La capital de Francia es París**

París es una d...

Turno 3:
  👤 Human: ¿Y cuántos habitantes tiene?
  🤖 AI: **Población de París**

Según los datos más recien...

Turno 4:
  👤 Human: ¿Cómo me llamo?
  🤖 AI: **Tu nombre es Carlos**

Me lo dijiste al principi...



### Gestión de memoria: Sliding Window

Para conversaciones largas, mantener todo el historial es costoso. Una solución es usar una ventana deslizante.

In [8]:
class ChatbotWithSlidingWindow:
    """Chatbot with sliding window memory."""
    
    def __init__(self, system_prompt: str = None, max_history: int = 5):
        self.llm = ChatGroq(
            model_name="llama-3.3-70b-versatile",
            temperature=0.7
        )
        
        self.system_prompt = system_prompt or "Eres un asistente amable."
        self.history = []
        self.max_history = max_history  # Maximum number of turns to remember
    
    def respond(self, user_input: str) -> str:
        """Generate response with sliding window memory."""
        messages = [SystemMessage(content=self.system_prompt)]
        
        # Only include last N turns
        recent_history = self.history[-self.max_history:]
        
        for human_msg, ai_msg in recent_history:
            messages.append(HumanMessage(content=human_msg))
            messages.append(AIMessage(content=ai_msg))
        
        messages.append(HumanMessage(content=user_input))
        
        response = self.llm.invoke(messages)
        ai_response = response.content
        
        self.history.append((user_input, ai_response))
        
        return ai_response

# Test sliding window
sliding_bot = ChatbotWithSlidingWindow(max_history=3)
print(f"Bot con memoria de {sliding_bot.max_history} turnos")

Bot con memoria de 3 turnos


<a name="streamlit"></a>
## 5. Interfaces con Streamlit

**Streamlit** permite crear interfaces web interactivas fácilmente. A continuación se muestra el código para una aplicación de chatbot.

**Nota:** Este código debe ejecutarse como script de Python, no en Jupyter.

In [ ]:
# This is the code for a Streamlit chatbot app
# Save as 'chatbot_app.py' and run with: streamlit run chatbot_app.py

streamlit_code = '''
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import os

# Page configuration
st.set_page_config(
    page_title="Chatbot con LLM",
    page_icon="🤖",
    layout="centered"
)

st.title("🤖 Chatbot con LLM")

# Sidebar for API key
with st.sidebar:
    st.header("Configuración")
    api_key = st.text_input("GROQ API Key", type="password")
    
    if api_key:
        os.environ["GROQ_API_KEY"] = api_key
    
    if st.button("Limpiar historial"):
        st.session_state.messages = []
        st.rerun()

# Initialize session state for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Chat input
if prompt := st.chat_input("Escribe tu mensaje..."):
    if not api_key:
        st.error("Por favor, introduce tu API key en la barra lateral.")
    else:
        # Add user message to history
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        with st.chat_message("user"):
            st.write(prompt)
        
        # Generate response
        with st.chat_message("assistant"):
            with st.spinner("Pensando..."):
                try:
                    llm = ChatGroq(
                        model_name="llama-3.3-70b-versatile",
                        temperature=0.7
                    )
                    
                    # Build messages
                    messages = [SystemMessage(content="Eres un asistente amable.")]
                    
                    for msg in st.session_state.messages:
                        if msg["role"] == "user":
                            messages.append(HumanMessage(content=msg["content"]))
                        else:
                            messages.append(AIMessage(content=msg["content"]))
                    
                    response = llm.invoke(messages)
                    st.write(response.content)
                    
                    # Add assistant response to history
                    st.session_state.messages.append({
                        "role": "assistant", 
                        "content": response.content
                    })
                    
                except Exception as e:
                    st.error(f"Error: {e}")
'''

print("Código para Streamlit Chatbot:")
print("="*50)
print("Guarda este código como 'chatbot_app.py'")
print("Ejecuta con: streamlit run chatbot_app.py")
print("="*50)
print(streamlit_code)

In [ ]:
# Save the Streamlit app to a file
with open('chatbot_app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print("✓ Archivo 'chatbot_app.py' creado.")
print("Para ejecutar: streamlit run chatbot_app.py")

<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio 1: Chatbot FAQ personalizado

Mejora el chatbot basado en reglas para manejar más casos.

In [ ]:
# Exercise 1: Create a more sophisticated rule-based chatbot
# Add rules for:
# - Product returns
# - Payment methods
# - Shipping information
# - Opening hours

class EnhancedRuleChatbot:
    def __init__(self):
        self.rules = {
            # Add your rules here
            'hola': '¡Hola! ¿En qué puedo ayudarte?',
            # 'devolucion': '...',
            # 'pago': '...',
            # etc.
        }
        self.default_response = "No entendí. ¿Puedes reformular?"
    
    def respond(self, user_input: str) -> str:
        user_input_lower = user_input.lower()
        for keyword, response in self.rules.items():
            if keyword in user_input_lower:
                return response
        return self.default_response

# Test your chatbot
# enhanced_bot = EnhancedRuleChatbot()
# print(enhanced_bot.respond("¿Cómo puedo devolver un producto?"))

### Ejercicio 2: Chatbot con personalidad

Crea un chatbot con una personalidad específica.

In [ ]:
# Exercise 2: Create a chatbot with a specific personality
# Ideas:
# - A pirate who speaks in pirate language
# - A Yoda-like character
# - A formal butler
# - A cheerful fitness coach

personality_prompt = """
# Define your personality prompt here
# Include:
# - Character description
# - Speaking style
# - Typical phrases
# - Topics they're expert in
"""

# personality_bot = LLMChatbot(system_prompt=personality_prompt)
# Test with some messages

### Ejercicio 3: Chatbot híbrido

Combina reglas con LLM: usa reglas para casos conocidos y LLM para el resto.

In [ ]:
# Exercise 3: Hybrid chatbot (rules + LLM)

class HybridChatbot:
    """Chatbot that uses rules for known cases and LLM for others."""
    
    def __init__(self):
        # Define rules for FAQ
        self.rules = {
            'horario': 'Horario: L-V 9:00-18:00',
            'precio': 'Consulta precios en www.ejemplo.com/precios',
            # Add more rules...
        }
        
        # LLM for complex questions
        self.llm = ChatGroq(
            model_name="llama-3.3-70b-versatile",
            temperature=0.7
        )
    
    def respond(self, user_input: str) -> str:
        # First, try rules
        user_input_lower = user_input.lower()
        for keyword, response in self.rules.items():
            if keyword in user_input_lower:
                return f"[REGLA] {response}"
        
        # If no rule matches, use LLM
        messages = [
            SystemMessage(content="Eres un asistente de tienda online."),
            HumanMessage(content=user_input)
        ]
        response = self.llm.invoke(messages)
        return f"[LLM] {response.content}"

# Test hybrid chatbot
# hybrid = HybridChatbot()
# print(hybrid.respond("¿Cuál es el horario?"))  # Should use rule
# print(hybrid.respond("¿Qué opinas del cambio climático?"))  # Should use LLM

## Resumen

En este notebook hemos aprendido:

1. **Chatbots basados en reglas**: Simples pero limitados
2. **Chatbots con LLM**: Inteligentes y flexibles
3. **Memoria conversacional**: Fundamental para coherencia
4. **Sliding window**: Gestión eficiente de memoria
5. **Streamlit**: Interfaces web interactivas

### Tipos de memoria en chatbots

| Tipo | Descripción | Uso |
|------|-------------|-----|
| Sin memoria | Cada mensaje es independiente | Consultas simples |
| Memoria completa | Guarda todo el historial | Conversaciones cortas |
| Sliding window | Guarda últimos N turnos | Conversaciones largas |
| Resumen | Guarda resumen de la conversación | Conversaciones muy largas |

En el siguiente notebook veremos **Vector Stores y Retrieval**, fundamentales para crear chatbots con acceso a información externa.

---

## Referencias

- [LangChain Chat Models](https://python.langchain.com/docs/modules/model_io/chat/)
- [Streamlit Documentation](https://docs.streamlit.io/)
- [Groq Console](https://console.groq.com/)